# Movie_review_sentiment_Analysis

**Business Problem:** Create a model which Predict the moview review sentiment positive/negative on given Comments form User.

**Dataset location :** [http://www.cs.cornell.edu/people/pabo/movie-review-data/](http://www.cs.cornell.edu/people/pabo/movie-review-data/)

This is the dataset from cornell which contain positive and negative review about movie. this is the text classification basic problem.

so lets get started

we will import basis libraries require for analysis of dataset like

1. numpy
2. re
3. nltk
4. pandas
5. sklearn

In [15]:
#import libraries
import numpy as np
import nltk
import re
import pickle
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from textblob import TextBlob
import warnings

#sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from gensim.models import Word2Vec
from nltk.corpus import stopwords
import gensim

In [16]:
warnings.filterwarnings('ignore')

1. from load_files we can load all file present in folder, it can load even from folder
2. we will store data which we are using for training in X and target variable in Y
3. now lets create padans dataframe from X and Y
4. lets print first few rows of dataset to take overall look
5. in dataset, almost 2000 rows are available

In [17]:
#load files, 
review_rating=load_files('txt_sentoken/')
X,Y= review_rating.data,review_rating.target
data={'comments':X,'review':Y}
data=pd.DataFrame(data)
length=len(X)
data.head(5)

,comments,review
0,arnold schwarzenegger has been an icon for act...,0
1,good films are hard to find these days . \ngre...,1
2,quaid stars as a man who has taken up the prof...,1
3,we could paraphrase michelle pfieffer's charac...,0
4,kolya is one of the richest films i've seen in...,1


In [18]:
data.shape

(2000, 2)

now we are applying some text cleaning procedure on dataset , so before doing changes we will take copy as pickle of dataset and work on pickle dataset

In [19]:
#dump and load file from disk
with open('X.pickle','wb') as f:
    pickle.dump(X,f)
    
with open('Y.pickle','wb') as f:
    pickle.dump(Y,f)
    
x1=open('X.pickle','rb')
y1=open('Y.pickle','rb')

x=pickle.load(x1)
y=pickle.load(y1)

1. Lets look at the some of the random comments of user, it contain noisy data, we have to remove those noisy data to make data for training purpose
2. we use python re library for cleaning and preprocessing data

In [20]:
#preprocess data using reguler expression
data=[]
for i in range (len(x)):
    review_rating=re.sub(r'\W',' ',X[i])
    review_rating=review_rating.lower()
    review_rating=re.sub(r'\s[a-z]\s+',' ',review_rating)
    review_rating=re.sub(r'^[a-z]\s+',' ',review_rating)
    review_rating=re.sub(r'\s+',' ',review_rating)
    data.append(review_rating)

1. data is available in text format and now we need to convert it into vector form
2. this dataset is smal in size, so we can directly apply tfidf vectorizer to convert text to vector form

In [22]:
#TFIDF applying
tfidf=TfidfTransformer()
#transformer = TfidfTransformer()
X = tfidf.fit_transform(X).toarray()

tfidfv=TfidfVectorizer(max_features =length, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X=tfidfv.fit_transform(data).toarray()


1. now we are good to split the dataset into training and testing set, we split it into 70:30 ratio

In [23]:
#split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.30, random_state = 7)

1. we dont know which algorithm is best suite fro this problem,
2. we have list of classifier algorithm which we are going to apply on dataset
3. it's look like LogisitcRegression  gives the best accuracy score almost ~83%
4. Now we will tune the logisitc regression for achieving better results

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.30, random_state = 7)

In [25]:
model2=[]
model2.append(('tfidfLogR',Pipeline([('LogR',LogisticRegression())])))
model2.append(('tfidfSVC',Pipeline([('SVC',SVC())])))
model2.append(('tfidfNB',Pipeline([('NB',MultinomialNB())])))
model2.append(('tfidfDC',Pipeline([('DC',DecisionTreeClassifier())])))
model2.append(('tfidfKNN',Pipeline([('KNN',KNeighborsClassifier())])))
model2.append(('tfidfdAdaB',Pipeline([('AdaB',AdaBoostClassifier())])))
model2.append(('tfidfdGB',Pipeline([('GB',GradientBoostingClassifier())])))
model2.append(('tfidfdRFR',Pipeline([('RFR',RandomForestClassifier())])))
model2.append(('tfidfdETR',Pipeline([('ETR',ExtraTreesClassifier())])))

names=[]
result=[]
for name , algo in model2:
    kfold=KFold(n_splits=10, random_state=7)
    cv_result=cross_val_score(algo,X_train,Y_train,scoring='accuracy',cv=kfold,verbose=0,n_jobs=-1)
    result.append(cv_result)
    names.append(name)
    print(name,cv_result.mean()*100,cv_result.std())

('tfidfLogR', 82.928571428571431, 0.039311679730905769)
('tfidfSVC', 51.571428571428577, 0.033775972621533965)
('tfidfNB', 80.357142857142847, 0.043594841484763239)
('tfidfDC', 62.214285714285708, 0.043501114224528896)
('tfidfKNN', 65.214285714285708, 0.053361493416056456)
('tfidfdAdaB', 75.928571428571416, 0.032269591423075755)
('tfidfdGB', 77.785714285714278, 0.033571428571428578)
('tfidfdRFR', 68.928571428571431, 0.04740726716012892)
('tfidfdETR', 68.5, 0.029005629291781382)


In [26]:

#C_val=np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.2,1.5,1.8,2.0])
multi_class =['ovr', 'auto']
solver =['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
par_grid=dict(multi_class=multi_class,solver=solver)
fitmodel=LogisticRegression()
kfold=KFold(n_splits=10, random_state=7)
grid=GridSearchCV(estimator=fitmodel,param_grid=par_grid,scoring='accuracy',cv=kfold)
grid_result=grid.fit(X_train,Y_train)
print("Best result for gridSearch",grid_result.best_score_,grid_result.best_params_)

('Best result for gridSearch', 0.82928571428571429, {'multi_class': 'ovr', 'solver': 'newton-cg'})


1. still after tuning , we got the same results, now we will train our model on Logisitc regresion and predict the feauture review


In [27]:

model=LogisticRegression()
model.fit(X_train,Y_train)
decision=model.predict(X_test)
finalScore=accuracy_score(decision,Y_test)
finalScore1=confusion_matrix(decision,Y_test)
finalScore2=classification_report(decision,Y_test)
print(finalScore)
print(finalScore1)
print(finalScore2)

0.861666666667
[[249  54]
 [ 29 268]]
              precision    recall  f1-score   support

           0       0.90      0.82      0.86       303
           1       0.83      0.90      0.87       297

   micro avg       0.86      0.86      0.86       600
   macro avg       0.86      0.86      0.86       600
weighted avg       0.86      0.86      0.86       600



1. after training and prediction on testing dataset. wew got almost 86% accuracy score 
2. confusion_score also look good